In [6]:
from pathlib import Path
import sys
import os

root = Path.cwd().parent
sys.path.append(str(root))

In [7]:
icl_path = root / "data" / "bsc_dataset_regplans" / "icl_examples.txt"

with open(icl_path, "r", encoding="utf-8") as f:
    content = f.read()

all_ICL_examples = []

for example in content.split("---"):
    all_ICL_examples.append(example.strip())

In [8]:
from openai import OpenAI
from src.utils import get_openai_api_key

client = OpenAI(api_key=get_openai_api_key())

In [ ]:
import pandas as pd
from tqdm import tqdm
import time

def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

df = pd.DataFrame({
    'example_id': range(len(all_ICL_examples)),
    'text': all_ICL_examples
})

embeddings = []
for i, text in tqdm(enumerate(df['text']), total=len(df)):
    try:
        embedding = get_embedding(text)
        embeddings.append(embedding)
        
        time.sleep(0.1)
        
    except Exception as e:
        print(f"Error with example {i}: {e}")
        embeddings.append(None)

df['embedding'] = embeddings

d = len(df['embedding'][0])

output_path = root / "data" / "bsc_dataset_regplans" / f"embedded_icl_examples_{d}d.csv"
df.to_csv(output_path, index=False)

100%|██████████| 2373/2373 [16:31<00:00,  2.39it/s]
